In [1]:
import numpy as np
import re
from tqdm import tqdm_notebook as tqdm
import time

with open('../data/wiki_short.txt', 'r') as f:
    lines = f.read().lower().splitlines()
    
import example

N = 100
M = 10
n = 0

In [2]:
list_of_fit_times = []
############################################

iterable = tqdm(range(M))
for _ in iterable:
    start = time.time()
    ret = example.vectorize_para("\w\w+", lines[n:N], 1, 1)
    end = time.time()
    
    list_of_fit_times.append(end-start)
    
    iterable.set_postfix_str('(fit={})'.format(np.mean(list_of_fit_times)))

print('(fit={})'.format(np.mean(list_of_fit_times)))

NameError: name 'tqdm' is not defined

In [7]:
list_of_fit_times = []
############################################

iterable = tqdm(range(M))
for _ in iterable:
    start = time.time()
    ret = example.vectorize("\w\w+", lines[n:N])
    end = time.time()
    
    list_of_fit_times.append(end-start)
    
    iterable.set_postfix_str('(fit={})'.format(np.mean(list_of_fit_times)))

print('(fit={})'.format(np.mean(list_of_fit_times)))

/home/local/AP-TEAM/grabovoy/WORKS/InternetDate/python_torch/lib/python3.6/site-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.



(fit=0.0017291545867919923)
